# Robust Portfolio Optimization
We use the returns of six portfolios formed on size and book-to-market (2$\times$3) obtained from Kenneth French's website. 
We choose monthly returns from 1984 January to 2014 January. 
This gives us a total of $N=360$ return realizations for each of the six portfolios.

Let $\mathbf{a}\in \mathbb{R}^6$ be an investment strategy on the six portfolios. 
We assume that the initial wealth level at time $t_0=0$ is $W_0=5$ and one does not buy or burrow more than the initial wealth, i.e. $|a_i|\leq 5$. Let $\mathbf{R}$ be a $360\times 6$ matrix with $\mathbf{r}_1,\ldots,\mathbf{r}_{360}\in \mathbb{R}^6$ the vectors of realisations of returns for all six portfolios. For each realisation $\mathbf{r}_i$, the wealth level after a single period is defined as $W_0(1+\mathbf{a}^T\mathbf{r}_i)=5(1+\mathbf{a}^T\mathbf{r}_i)$. We consider the following worst case risk evaluation minimization problem 
\begin{align}
\begin{split}
\min_{\mathbf{a}\in \mathbb{R}^{6},|a_i|\leq 5, c\in \mathbb{R}}\{c|\sup_{(\mathbf{q},\bar{\mathbf{q}})\in U_{\phi,h}}-\sum^{360}_{i=1}\bar{q}_iu(5(1+\mathbf{a}^T\mathbf{r}_i))\leq c, \mathbf{a}^T\mathbf{1}=1\},   
\end{split}
\end{align}
where we choose the modified chi-squared function $\phi(x)=(x-1)^2$ with radius $r=\frac{\phi''(1)}{2N}\chi^2_{d,0.95}$ and $d=N-1=359$. Furthermore, we choose $h(p)=1-(1-p)^2,~p\in[0,1]$ as the distortion function and $u(x)=1-e^{-x/\lambda}$ with $\lambda=10$ the exponential utility function.
The solution of this problem will be compared to the nominal risk evaluation minimization problem, where the nominal probability is the uniform distirbution $p_i=\frac{1}{360}$.
\begin{align}
\begin{split}
\min_{\mathbf{a}\in \mathbb{R}^6,|a_i|\leq 5, c\in \mathbb{R}}\{c|&\sup_{\bar{\mathbf{q}}\in M_h(\mathbf{p})}-\sum^{360}_{i=1}\bar{q}_iu(5(1+\mathbf{a}^T\mathbf{r}_i))\leq c, \mathbf{a}^T\mathbf{1}=1\}.    
\end{split}
\end{align}

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
import matplotlib.pyplot as plt
import datetime as date
from datetime import datetime as dt
from dateutil.relativedelta import *
import scipy.stats
from scipy.stats import rankdata
import distortion_function as hf
import phi_divergence as phi
import affine_approx as af
import Utility_functions as ut
import Cutting_plane as ct
from time import process_time
import Hit_and_Run as hr

In [2]:
### Reading the Kenneth-French return data's
df_returns6 = pd.read_csv('6_Portfolios_2x3.csv', skiprows = 15) 
df_returns = df_returns6[0:1144].copy()
df_returns['Date'] = pd.to_datetime(df_returns['Date'], format = '%Y%m')
for i in range(1, len(df_returns.columns)):
    df_returns[df_returns.columns[i]] = pd.to_numeric(df_returns[df_returns.columns[i]])
df_returns

,Date,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
0,1926-07-01,1.0874,0.9349,-0.0695,5.7168,1.9620,1.4222
1,1926-08-01,0.7030,1.2300,5.3842,2.7154,2.6930,6.3154
2,1926-09-01,-2.9117,-0.1303,-0.4374,1.4287,0.0704,-0.7967
3,1926-10-01,-3.8196,-4.5860,-2.0112,-3.5898,-2.3398,-4.0970
4,1926-11-01,3.1806,3.7233,2.0944,3.1292,2.8952,3.4614
...,...,...,...,...,...,...,...
1139,2021-06-01,5.6058,0.4400,-1.0979,4.8188,-1.2594,-4.0036
1140,2021-07-01,-5.5593,-1.8623,-3.6521,3.1048,-0.0099,-2.3000
1141,2021-08-01,2.3903,1.5124,2.6680,3.5667,1.4122,3.0371
1142,2021-09-01,-4.3421,-3.4661,0.6445,-5.4525,-3.8570,-0.2526


In [3]:
### Selecting the return values between 1984 and 2014, since everything was in percentage, we divde the return by 100
startdate = dt(1984,1,3)
enddate = dt(2014,1,3)
X = df_returns[np.logical_and(df_returns.Date >= startdate, df_returns.Date <= enddate )][df_returns.columns[1:7]]
X = X.reset_index(drop = True)
R = X.to_numpy()     
R = R/100
#### The risk free rate is taken to be 0.07%
r_f = 0.0007  
W0 = 5

We specify the $\phi$, $h$ and $u$ functions.

In [4]:
phi_func = phi.mod_chi2_cut
phi_conj = phi.mod_chi2_conj
phi_dot = 2

h_conj = hf.h_spw_conj
h_func = hf.h_spw_cut
h_eva = hf.h_spw_eva
### parameter of h function, in this case we have h(p)=1-(1-p)^2, hence we take par = 2
h_par = 2 

utility = ut.exp_utility
utility_eva = ut.exp_utility_eva

### Defining the nominal probability vector and denote I the number of assets
N=R.shape[0]
p = np.zeros(N)+1/N
I = R.shape[1]


### defining phi-divergence uncertainty set radius
alpha_phi_set = 0.95
r = phi_dot/(2*N)*scipy.stats.chi2.ppf(alpha_phi_set, N-1)   

The worst case risk evaluation minimization problem will be solved using the piecewise-linear approximation method and the rank-induced robust counterpart algorithm. The former yields a lower bound and the latter yields an upper bound. To initiate the piecewise-linear approximation method, we need to obtain the slopes and the constants that constitutes the piecewise linear approximation.

In [5]:

### The precision of the PL approximation
e_pl = 0.001       

### This gives the slope and constants that constitutes the PL approximation of h(p)=1-(1-p)^2
x_points = af.affine_approx_hspw(h_par,e_pl)     
[slope, const] = af.makepoints(af.sing_pw,x_points,par = h_par)  
                                                               

In [6]:
###The following function can calculate the approximated solution using the rank-induced robust counterpart algorithm  

def RC_exputility_pmin(sets,p,R,r,phi_conj, h_conj,W0,par=2,par_u=1):
    N = len(p)
    I = len(R[0])
    M = len(sets)
    c = cp.Variable(1)
    v = cp.Variable(M)
    lbda = cp.Variable(M, nonneg = True)
    a = cp.Variable(I)
    alpha = cp.Variable(1)
    beta = cp.Variable(1)
    gamma = cp.Variable(1,nonneg = True)
    t = cp.Variable(N)
    z = cp.Variable(M)
    s = cp.Variable(N)
    w = cp.Variable(N)
    constraints=[cp.abs(a)<=W0, cp.sum(a)==1]
    for i in range(N):
        lbdasum = 0
        vsum = 0
        for j in range(M):
            if i in sets[j]:
                lbdasum = lbdasum + lbda[j]
                vsum = vsum + v[j]
        arg = -W0*(1+(R @ a)[i])/par_u
        constraints.append(-(1-cp.exp(arg)) - lbdasum - beta <= 0)
        constraints.append(s[i] == -alpha + vsum)
        constraints = phi_conj(gamma,s[i],t[i],w[i],constraints)
    constraints = h_conj(lbda,v,z,par,constraints)
    constraints.append(alpha + beta + gamma * r  + cp.sum(z) + p@t <= c)
    obj = cp.Minimize(c)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(a.value, prob.value)


### The following function can calculate the approximated solution using the picewise-linear approximation method

def af_RC_exp_pmin(p,R,r,phi_conj,slope,const,W0,par = 2, par_u = 1):
    N = len(p)
    I = len(R[0])
    K = len(slope)
    c = cp.Variable(1)
    lbda = cp.Variable((N,K), nonneg = True)
    a = cp.Variable(I)
    v = cp.Variable(K, nonneg = True)
    alpha = cp.Variable(1)
    beta = cp.Variable(1)
    gamma = cp.Variable(1,nonneg = True)
    t = cp.Variable(N)
    s = cp.Variable(N)
    w = cp.Variable(N)
    constraints = [cp.abs(a)<= W0, cp.sum(a) == 1]
    for i in range(N):
        arg = -W0*(1+(R @ a)[i])/par_u
        constraints.append(-(1-cp.exp(arg)) - cp.sum(lbda[i]) - beta <= 0)
        constraints.append(s[i] == -alpha + lbda[i]@slope)
        constraints.append(lbda[i] <= v)
        constraints = phi_conj(gamma,s[i],t[i],w[i],constraints)
    constraints.append(alpha + beta + gamma * r  + v@const + p@t <= c)
    obj = cp.Minimize(c)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(a.value, prob.value)


In [23]:
t1_af_res = process_time()
af_res = af_RC_exp_pmin(p,R,r,phi_conj,slope,const,W0,par = h_par, par_u = 10)
t2_af_res = process_time()
### the solution and objective value of the piecewise-linear approximation method
print(af_res)   
### Total run time of the algorithm
print('time', t2_af_res- t1_af_res, 'seconds')

(array([-1.55766188,  2.31243927, -0.39538981,  1.23519695, -0.18097537,
       -0.41360916]), -0.37873350459491345)
time 5.265625 seconds


In [15]:
w_af = af_res[0]
### determining the rank of the pl solution to initiate rank-induced rc algorithm
rank = np.argsort(W0*(1+R.dot(w_af)))    
sets = ct.ranktoset(rank)
t1_rc_res = process_time()
rc_res = RC_exputility_pmin(sets,p,R,r,phi_conj, h_conj,W0,par=h_par,par_u=10)
t2_rc_res = process_time()
### the solution and objective value of the rank-induced robust counterpart algorithm
print(rc_res)      
print('time', t2_rc_res - t1_rc_res, 'seconds')

(array([-1.55763529,  2.31217721, -0.3951753 ,  1.23521153, -0.18086841,
       -0.41370974]), -0.3786601778588288)
time 35.84375 seconds


The nominal solution can be approximately calculated using the piecewise-linear approximation method and set the phi-divergence radius $r$ to zero.

In [19]:
### r = 0 is set here in the argument
res_nom = af_RC_exp_pmin(p,R,0,phi_conj,slope,const,W0,par = h_par, par_u = 10)   
print(res_nomaf)

(array([-1.90771996,  2.55136958,  0.09114196,  1.45125434, -0.6440066 ,
       -0.54203934]), -0.3921887272593855)


Alternatively, the nominal solution can also be calculated exactly by using the cutting-plane algorithm and set the precision of the cutting-plane parameter $\epsilon_{tol}$ to zero. However, this method requires many iterations.

In [ ]:
tb = process_time()
### eps_tol =0 is set here in the argument
res_nom = ct.cut_nom_pmin(R,p,0,utility,utility_eva,h_eva,W0,par=h_par,par_u=10)  
te = process_time()
print('time', te-tb, 'seconds')

In [21]:
#### Calculating the worst case risk of the nominal solution
w_nom = res_nom[0]       
x_nom = ut.exp_utility_eva(R,w_nom,W0,10)
print(ct.robustcheck(x_nom,p,h_func,phi_func,r,h_par)[0])

-0.37794744914733636


In [22]:
### Calculating the nominal risk of the robust solution, which is taken as the solution of rc_res
w_rob = rc_res[0]
x_rob = ut.exp_utility_eva(R,w_rob,W0,10)
[risk_robnom, qb_nomrb]=ct.nominal_risk(x_rob,p,h_eva,h_par) 
print(risk_robnom)    

-0.3918807225379179


Consider now a portfolio return maximization problem under a worst case risk evaluation constraint. Using nearly the same parameters setting as in the previous problem (except for we set $W_0=100$, $\lambda=200$ so that we can get higher return), we aim to solve the robust problem
\begin{align}\label{robust_problem_experiment}
    \begin{split}
        \max_{\mathbf{a}\in \mathbb{R}^6: |a_i|\leq 100}&\sum^{360}_{k=1}p_ku(5(1+\mathbf{a}^T\mathbf{r}_k+(1-\mathbf{a}^T\mathbf{1})r_f))\\
        \text{subject to}&~ \sup_{(\mathbf{q},\bar{\mathbf{q}})\in U_{\phi,h}}-\sum^{360}_{k=1}\bar{q}_ku(5(1+\mathbf{a}^T\mathbf{r}_k+(1-\mathbf{a}^T\mathbf{1})r_f))\leq c.
    \end{split}
\end{align}
The nominal problem is defined as
\begin{align}\label{nominal_experiment}
    \begin{split}
        \max_{\mathbf{a}\in \mathbb{R}^6: |a_i|\leq 100}&\sum^{360}_{k=1}p_ku(5(1+\mathbf{a}^T\mathbf{r}_k+(1-\mathbf{a}^T\mathbf{1})r_f))\\
        \text{subject to}&~ \sup_{\bar{\mathbf{q}}\in M_h(\mathbf{p})}-\sum^{360}_{k=1}\bar{q}_ku(5(1+\mathbf{a}^T\mathbf{r}_k+(1-\mathbf{a}^T\mathbf{1})r_f))\leq c.
    \end{split}
\end{align}

In [14]:
#### Changing the utility function code that is suitable for this maximization problem
utility = ut.exp_utility_pmax
utility_eva = ut.exp_utility_eva_pmax
### Set the risk evaluation constraint c = 0.02
c = 0.02
W0 = 100

In [8]:
#### Re-initialize the slope and constants of the piecewise-linear approximations
e_pl = 0.001 
x_points = af.affine_approx_hspw(h_par,e_pl)     
[slope, const] = af.makepoints(af.sing_pw,x_points,par = h_par)

In [12]:
### These functions uses the piecewise linear approximation method and the rank-induced robust counterpart method for
### the maximization problems


def RC_exputility_pmax(sets,p,R,r,c,r_f,phi_conj, h_conj,W0,par=2,par_u=1):
    N = len(p)
    I = len(R[0])
    M = len(sets)
    v = cp.Variable(M)
    lbda = cp.Variable(M, nonneg = True)
    a = cp.Variable(I)
    alpha = cp.Variable(1)
    beta = cp.Variable(1)
    gamma = cp.Variable(1,nonneg = True)
    t = cp.Variable(N)
    z = cp.Variable(M)
    s = cp.Variable(N)
    w = cp.Variable(N)
    constraints=[cp.abs(a)<=W0]
    f_obj = 0
    for i in range(N):
        lbdasum = 0
        vsum = 0
        for j in range(M):
            if i in sets[j]:
                lbdasum = lbdasum + lbda[j]
                vsum = vsum + v[j]
        arg = -(W0*(1+(R @ a)[i]+(1-cp.sum(a))*r_f))/par_u
        f_obj = (1-cp.exp(arg))*p[i] + f_obj
        constraints.append(-(1-cp.exp(arg)) - lbdasum - beta <= 0)
        constraints.append(s[i] == -alpha + vsum)
        constraints = phi_conj(gamma,s[i],t[i],w[i],constraints)
    constraints = h_conj(lbda,v,z,par,constraints)
    constraints.append(alpha + beta + gamma * r  + cp.sum(z) + p@t <= c)
    obj = cp.Maximize(f_obj)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(a.value, prob.value)

def af_RC_exp_pmax(p,R,r,r_f,c,phi_conj,slope,const,W0,par = 2, par_u = 1):
    N = len(p)
    I = len(R[0])
    K = len(slope)
    lbda = cp.Variable((N,K), nonneg = True)
    a = cp.Variable(I)
    v = cp.Variable(K, nonneg = True)
    alpha = cp.Variable(1)
    beta = cp.Variable(1)
    gamma = cp.Variable(1,nonneg = True)
    t = cp.Variable(N)
    s = cp.Variable(N)
    w = cp.Variable(N)
    constraints = [cp.abs(a)<= W0]
    f_obj = 0
    for i in range(N):
        arg = -(W0*(1+(R @ a)[i]+(1-cp.sum(a))*r_f))/par_u
        f_obj = (1-cp.exp(arg))*p[i] + f_obj
        constraints.append(-(1-cp.exp(arg)) - cp.sum(lbda[i]) - beta <= 0)
        constraints.append(s[i] == -alpha + lbda[i]@slope)
        constraints.append(lbda[i] <= v)
        constraints = phi_conj(gamma,s[i],t[i],w[i],constraints)
    constraints.append(alpha + beta + gamma * r  + v@const + p@t <= c)
    obj = cp.Maximize(f_obj)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(a.value, prob.value)

In [15]:
t1_af = process_time()
##### Obtaining an upper bound of using piecewise-linear approximation
af_pmax_res = af_RC_exp_pmax(p,R,r,r_f,c,phi_conj,slope,const,W0,par = h_par, par_u = 200)
t2_af = process_time()
print(af_pmax_res)
print('seconds', t2_af-t1_af)
t1_af_l = process_time()
#### Obtaining a lower bound using piecewise-linear approximation, by adding the error term e_pl to the constants
af_pmax_res_l = af_RC_exp_pmax(p,R,r,r_f,c,phi_conj,slope,const+e_pl,W0,par = h_par, par_u = 200)
t2_af_l = process_time()
print(af_pmax_res_l)
print('seconds', t2_af_l-t1_af_l)
#### Obtaining a lower bound using the rank-induced robust counterpart algorithm
w_af_rob = af_pmax_res[0]
rank = np.argsort(W0*(1+R.dot(w_af_rob)+(1-np.sum(w_af_rob))*r_f))
sets = ct.ranktoset(rank)
t1_resct = process_time()
rc_robmax = RC_exputility_pmax(sets,p,R,r,c,r_f,phi_conj, h_conj,W0,par=2,par_u=200)
t2_resct = process_time()
print(rc_robmax)
print('res-time',t2_resct-t1_resct, 'seconds')

(array([-41.45716   ,  55.81420784,   0.44551949,  26.61893252,
       -16.10308739, -10.69012328]), 0.44772711598820836)
seconds 6.078125
(array([-41.34189433,  55.65820961,   0.43623302,  26.52841499,
       -16.06510988, -10.64736032]), 0.44770022010098304)
seconds 6.140625
(array([-41.36258815,  55.69373123,   0.4380042 ,  26.54956699,
       -16.07116502, -10.6581975 ]), 0.4477091583929269)
res-time 34.328125 seconds


Alternatively, one can compute a solution using the cutting-plane and the rank-indcued robust counterpart algorithms

In [16]:
#### The cutting-plane algorithm with tolerance parameter e_tol
e_tol = 0.001 
t1_cutstart = process_time()
cut_res = ct.cut_rob_pmax(R,p,e_tol,utility,utility_eva,h_func,phi_func,h_eva,r, r_f, c, W0, par=h_par, par_u=200)
t1_cutstop = process_time()
print("cut-time:", t1_cutstop-t1_cutstart) 
print(cut_res)
#### Determining the rank of the cutting-plane solution to compute the rank-induced robust counterpart
w_cut = cut_res[0]
rank = np.argsort(W0*(1+R.dot(w_cut)+(1-np.sum(w_cut))*r_f))
sets = ct.ranktoset(rank)
t1_resct = process_time()
rc_resmax = RC_exputility_pmax(sets,p,R,r,c,r_f,phi_conj, h_conj,W0,par=h_par,par_u=200)
t2_resct = process_time()
print('res-time',t2_resct-t1_resct)
print(rc_resmax)

0.07754350224598186 0.02 0
0.020472059982278613 0.02 1
cut-time: 17.375
(array([-41.47371081,  55.83687808,   0.41341288,  26.59427596,
       -16.12152565, -10.65382054]), 0.4477138709398567, 1)
res-time 34.0
(array([-41.41842187,  55.7936817 ,   0.39746181,  26.56581941,
       -16.10659185, -10.64642667]), 0.44770904082638724)


The nominal solution and its worst case risk evaluation is calculated below by setting $r=0$

In [18]:
nom_afres = af_RC_exp_pmax(p,R,0,r_f,c,phi_conj,slope,const,W0,par = h_par, par_u = 200)
print(nom_afres)
w_af_nom = nom_afres[0]
nom_afres1 = af_RC_exp_pmax(p,R,0,r_f,c,phi_conj,slope,const+e_pl,W0,par = h_par, par_u = 200)
print(nom_afres1)
x_nommax = ut.exp_utility_eva_pmax(R,r_f,nom_afres1[0],W0,200)
print(ct.robustcheck(x_nommax,p,h_func,phi_func,r,h_par)[0])

(array([-43.86009335,  59.07676109,   0.60714823,  28.4956384 ,
       -16.92814972, -11.54146516]), 0.44799771243526954)
(array([-43.86013074,  59.07682119,   0.60718407,  28.49569954,
       -16.92813808, -11.54151869]), 0.4479977124330769)
0.0775450454974164
